# Movie recommendation system 
-------------------
MovieLens 1M Dataset: 1 million ratings from 6000 users on 4000 movies. <br>
https://grouplens.org/datasets/movielens/1m/

-----------
### LSH Cosine (sim hash) <br>

#### Cosine similarity: <br>
$
\text{cosine_sim}(u, v) = \frac{
\sum\limits_{i \in I_{uv}} r_{ui} \cdot r_{vi}}
{\sqrt{\sum\limits_{i \in I_{uv}} r_{ui}^2} \cdot
\sqrt{\sum\limits_{i \in I_{uv}} r_{vi}^2}
}
$

#### Accuracy Metric: <br>
RMSE (Root Mean Squared Error) 

#### KNN Basic: <br>
$\hat{r}_{ui}$ : Predicted rating of user u for item i <br>
User Based: <br>
$ \hat{r}_{ui} = \frac{
\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
{\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)} $


In [1]:
import pandas as pd
import numpy as np
import scipy as sp

## Pre-processing

In [2]:
users = {}
movies = set()
with open("ratings.dat", 'r') as f:
    lines = f.readlines()
    np.random.shuffle(lines)
    test_lines = lines[:20000] # training dataset
    lines = lines[20000:] # test dataset
    for line in lines:
        u, m, r, t = line.split('::')
        if int(u) not in users:
            users[int(u)] = []
        users[int(u)].append((int(m), float(r)))
        movies.add(int(m))

In [5]:
# mapping movie to new index starting from 0
new_movies = {}
for i, m in enumerate(movies):
    new_movies[int(m)] = i

In [51]:
# create sparse matrix of users and movies with ratings
num_users = len(users.keys())
rating_matrix = np.empty(num_users, dtype=np.object)

for user in users:
    rating_matrix[user-1] = np.zeros(len(users[user]), dtype="int, float")
    i = 0
    for m, r in sorted(users[user],key=lambda x:x[0]):
        nm = new_movies[m]
        rating_matrix[user-1][i] = (nm,r)
        i += 1

In [12]:
# create random vectors 
num_movies = len(movies)
rv_list = []

b=6 # number of bands
r=8 # number of tables

for i in range(b*r):
    random_vector = np.random.normal(size=num_movies)
    for j in range(len(random_vector)):
        if random_vector[j]<0:
            random_vector[j]=-1
        else:
            random_vector[j]=1
    rv_list.append(random_vector)

In [13]:
# create hash string table of users
hash_string = {}
user_ind = 1
for u in rating_matrix:
    hash_string[user_ind] = ""
    for i in range(b*r):
        s = 0
        for mv, rt in u:
            s += rt*rv_list[i][mv]
        if s<=0:
            hash_string[user_ind] += "0"
        else:
            hash_string[user_ind] += "1"
    user_ind +=1

In [15]:
# convert hash string to r times (number of tables) of bands in integer for each user
band = {}
for h in hash_string:
    s = 0
    e = b
    band[h] = []
    for i in range(r):
        band[h].append(int(hash_string[h][s:e],2))
        s += b
        e += b

In [17]:
# find each user's neighbors in each table
table = {}
for u in band:
    for i, t in enumerate(band[u]):
        if i not in table:
            table[i] = {}
        if t not in table[i]:
            table[i][t] = []
        table[i][t].append(u)

In [20]:
# find each user's neighbors
neighbors = {}
for user in users.keys():
    neighbors[user] = set()

for t in table:
    for b in table[t]:
        for user in table[t][b]:
            neighbors[user].update(table[t][b])

## Training

In [23]:
def cosine(a,b):
    i = 0
    j = 0
    s = 0
    p = 0
    q = 0
    while i<len(a) and j<len(b):
        if int(a[i][0])==int(b[j][0]):
            p+=(a[i][1]**2)
            q+=(b[j][1]**2)
            s+=(a[i][1]*b[j][1])
            i+=1
            j+=1
        elif int(a[i][0])>int(b[j][0]):
            j+=1
        else:
            i+=1
    if p==0 or q==0:
        return 0.0
    return s*1.0/(p**0.5 * q**0.5)

In [24]:
# calculate cosine similarity bewteen each neighbor and the user
cosine_neighbors = {}

for user in users.keys():
    cosine_neighbors[user] = []

for user in neighbors:
    for neighbor in neighbors[user]:
        cosine_neighbors[user].append((neighbor,cosine(rating_matrix[user-1], rating_matrix[neighbor-1])))
    cosine_neighbors[user] = sorted(cosine_neighbors[user], key=lambda x: x[1], reverse=True)

## Test

In [35]:
def checkRating(user, movie):
    for m, r in rating_matrix[user]:
        if m == movie:
            return r
    return 0

In [54]:
# calculate ratings of each movie (the average ratings of the neighbors who rated this movie) 
# for each user in test dataset

actual_value = []
prediction = []
c = 0
for tl in test_lines[:20000]:
    u, m, r, t = tl.split('::')
    num = 0
    den = 0
    k = 0
    for n, sim in cosine_neighbors[int(u)]:
        if k == 20:
            break
        nr = checkRating(n-1, int(m))
        if nr:
            num += nr*sim
            den += sim
            k += 1
    c+=1
    if c%1000==0:
        print(c," out of 20000 done")
    if k>0 and den>0:
        p = num/den
        prediction.append(p)
        actual_value.append(r)
    else:
        me = 0
        t = 0
        for m,r in rating_matrix[int(u)-1]:
            me+=r
            t+=1
        prediction.append(me*1.0/t)
        actual_value.append(r)

(1000, ' out of 20000 done')
(2000, ' out of 20000 done')
(3000, ' out of 20000 done')
(4000, ' out of 20000 done')
(5000, ' out of 20000 done')
(6000, ' out of 20000 done')
(7000, ' out of 20000 done')
(8000, ' out of 20000 done')
(9000, ' out of 20000 done')
(10000, ' out of 20000 done')
(11000, ' out of 20000 done')
(12000, ' out of 20000 done')
(13000, ' out of 20000 done')
(14000, ' out of 20000 done')
(15000, ' out of 20000 done')
(16000, ' out of 20000 done')
(17000, ' out of 20000 done')
(18000, ' out of 20000 done')
(19000, ' out of 20000 done')
(20000, ' out of 20000 done')


In [71]:
# RMSE
np.sqrt(np.mean((np.array(prediction) - np.array(actual_value, dtype=np.float))**2))

1.2955390181119637